In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSD
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import matplotlib.pyplot as plt
from freesasa import calcCoord
from MDAnalysis.topology.tables import vdwradii
import os
from scipy.signal import savgol_filter
from concurrent.futures import ProcessPoolExecutor

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
u = mda.Universe("centered.gro")
sds_atoms = u.select_atoms("resname SDS")
atom = u.select_atoms("name OS1")


In [3]:
class MoleculeSystem:
    
    def __init__(self, top, traj):
        self.file_name = os.path.splitext(os.path.split(traj)[1])[0]
        self.u = mda.Universe(top, traj)
        self.ag = None
        self.rms_data = None
        self.selection = None
        self.sasa_data = np.zeros((len(self.u.trajectory, )))
        self.gyr_data = None
        
    def get_sasa_data(self):
        return self.sasa_data
    
    def get_gyr_data(self):
        return self.gyr_data.timeseries[:, 0]
    
    def get_rms_data(self):
        return self.rms_data
    
    def select(self, sel):
        self.ag = self.u.select_atoms(sel)
        self.selection = sel
        
    def rms(self):
        if not self.ag:
            raise TypeError('Please make a selection with select()')
        
        R = RMSD(self.ag, self.ag)
        R.run()
        
        self.rms_data = R.results.rmsd.T
        return self
    
    @staticmethod
    def _radgyr(atomgroup, masses, total_mass=None):
        # coordinates change for each frame
        coordinates = atomgroup.positions
        center_of_mass = atomgroup.center_of_mass()

        # get squared distance from center
        ri_sq = (coordinates - center_of_mass) ** 2
        
        # sum the unweighted positions
        sq = np.sum(ri_sq, axis=1)
        sq_x = np.sum(ri_sq[:,[1,2]], axis=1) # sum over y and z
        sq_y = np.sum(ri_sq[:,[0,2]], axis=1) # sum over x and z
        sq_z = np.sum(ri_sq[:,[0,1]], axis=1) # sum over x and y

        # make into array
        sq_rs = np.array([sq, sq_x, sq_y, sq_z])

        # weight positions
        rog_sq = np.sum(masses * sq_rs, axis=1) / total_mass
        # square root and return
        return np.sqrt(rog_sq)
    
    def gyr(self):
        if not self.ag:
            raise TypeError('Please make a selection with select()')
        
        gyradius = AnalysisFromFunction(self._radgyr, self.ag, self.ag.masses, total_mass=np.sum(self.ag.masses))
        gyradius.run()
        
        self.gyr_data = gyradius.results

        return self
        
    def sasa(self):
        if not self.ag:
            raise TypeError('Please make a selection with select()')
        print('Calculating SASA...')
        radii = np.array([vdwradii[ag.type] for ag in self.ag])
#         self.sasa_data = np.zeros((len(self.u.trajectory, )))
        
        for ts in self.u.trajectory:
            
            ag = self.u.select_atoms(self.selection)
            coords = self.ag.positions.flatten()
            self.sasa_results = calcCoord(coords, radii)
            self.sasa_data[ts.frame] = self.sasa_results.totalArea()
        print('Done with SASA.')
        return self
    
    def _smooth(self, data, windows=5, order=2):
        return savgol_filter(data, windows, order)

    def plot(self, *args, which='rms', save=False, xlim=None, ylim=None, smooth=False, smooth_window=5, smooth_order=2, **kwargs):
        fig = plt.figure(figsize=(10, 8))
        
        if which == 'rms':
            fig_title = 'rms'
            data = self.rms_data[2] if not smooth else self._smooth(self.rms_data[2], smooth_window, smooth_order)
            plt.ylabel('RMS')
            plt.plot(self.rms_data[1], data, *args, **kwargs)
        elif which == 'gyr':
            fig_title = 'gyr'
            data = self.gyr_data.timeseries[:, 0] if not smooth else self._smooth(self.gyr_data.timeseries[:, 0], smooth_window, smooth_order)
            plt.ylabel('$R_{gyr}$')
            plt.plot(self.gyr_data.frames, data, *args, **kwargs)
        elif which == 'sasa':
            fig_title = 'sasa'
            data = self.gyr_data.timeseries[:, 0] if not smooth else self._smooth(self.gyr_data.timeseries[:, 0], smooth_window, smooth_order)
            plt.ylabel('$\AA^2$')
            plt.plot(self.sasa_data.frames, data, *args, **kwargs)
        
        plt.xlabel('Time (ns)')

        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.title(fig_title)
        if save:
            if os.path.exists('.'.join((self.file_name, 'pdf'))):
                ans = input('File exists. Overwrite? (y|n) ')
                if ans == 'n':
                    return
            plt.savefig('./' + '.'.join((self.file_name, 'pdf')))
        
        plt.show()


        

In [ ]:
if __name__ == '__main__':
    

# Calc SASA, Radius of Gyration and RMS

Create a MoleculeSystem object by providing your topology and trajectory files.

```python
s = MoleculeSystem('example.tpr', 'example.xtc')
```

Make a selection using .select() method

```python
s.select('resname whatever')
```

Then you can run either one of these methods to calculate the appropriate parameter

```python
s.sasa()  # may take considerable amount of time especially in your case with the huge system even with waters removed. 
s.rms()
s.gyr()
```

When calculations are finished, you can access the data using .get_sasa_data(), .get_rms_data(), .get_gyr_data() methods.
If you want to plot certain data, call .plot() method and pass 'rms', 'sasa' or 'gyr' to which kwarg to plot the corresponding data (you can use any matplotlib option in s.plot()).
```python
s.plot(which='sasa')
```

P.S. the code is a mess, I'll update it in future. It should work though. If you'll have questions, let me know.